# About Prompt
**RAG 프롬프트 구조**
- 지시사항(instruction)
- 질문(사용자 입력 질문)
- 문맥(검색된 정보)

# 2-1. Prompt Template

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
# env 파일이 안들어와서 강제 로드
import os

# .env 파일 강제 로드
dotenv_path = os.path.join(os.getcwd(), ".env")
load_dotenv(dotenv_path)

True

In [11]:
# LangSmith 추적을 설정 https://smith.langchain.com
!pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [12]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

### 방법 1. from_template() 메소드 사용하여 PromptTemplate 객체 생성

In [13]:
from langchain_core.prompts import PromptTemplate

# template 정의: {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [14]:
# country 변수에 값을 넣어서 문장을 생성할 수 있음
# prompt 생성 .format 메소드 이용하여 변수에 값 넣기
prompt = prompt.format(country = "대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [15]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드로 Prompt Template 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [16]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
- : 추가 유효성 검사를 위해 'input_variables'를 명시적으로 지정
- 이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하는 경우 예외를 발생시킴

In [17]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template = template,
    input_variables = ["country"]
)

prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [18]:
# prompt 생성
prompt.format(country = "대한민국")

'대한민국의 수도는 어디인가요?'

In [19]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template = template,
    input_variables = ["country1"],
    partial_variables = {
        "country2": "미국" #dictionary 형태로 partial_variables 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [20]:
prompt.format(country1 = "대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [21]:
prompt_partial = prompt.partial(country2 = "캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [22]:
prompt_partial.format(country1 = "대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [23]:
chain = prompt_partial | llm

In [24]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [25]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이고 호주의 수도는 캔버라입니다.'

### 'partial_variables': 부분 변수 채움
- 항상 공통된 방식으로 가져오고 싶은 변수가 있는 경우 ex) 날짜, 시간
- 항상 현재 날짜가 표시되기를 원하는 프롬프트가 있을 경우, 현재 날짜 반환 함수를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리함

In [26]:
from datetime import datetime

# 오늘 날짜 출력
datetime.now().strftime("%B %d")

'February 17'

In [27]:
# 날짜 반환 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [29]:
prompt = PromptTemplate(
    template = "오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables = {"today": get_today}, #dictionary 형태로 partial_variables 전달
)

In [30]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 February 17 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [31]:
# chain 생성
chain = prompt | llm

In [32]:
# chain 실행 후 결과 확인
print(chain.invoke(3).content)

1. Michael Jordan - 1963년 2월 17일
2. Ed Sheeran - 1991년 2월 17일
3. Paris Hilton - 1981년 2월 17일


In [33]:
print(chain.invoke({"today": "Oct 02", "n": 3}).content)

1. 윤아 (1989년 10월 02일)
2. 곽도원 (1983년 10월 02일)
3. 리버플레이트 (1981년 10월 02일)


### 파일로부터 template 읽어오기

In [38]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")
prompt

PromptTemplate(name='fruit_color', input_variables=['fruit'], template='{fruit}의 색깔은 무엇인가요?')

In [39]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(name='fruit_color', input_variables=['fruit'], template='{fruit}의 색깔은 무엇인가요?')

In [40]:
prompt.format(fruit="사과")

'사과의 색깔은 무엇인가요?'

In [41]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품


### ChatPromptTemplate

In [42]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?'))])

In [43]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [46]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        #role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 생성
messages = chat_template.format_messages(
    name = "랭체인봇", user_input = "당신의 이름은 무엇입니까?"
)

messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 랭체인봇입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름은 무엇입니까?')]

In [47]:
llm.invoke(messages).content

'제 이름은 랭체인봇입니다. 필요하신 도움이 있으면 언제든지 물어보세요!'

In [48]:
chain = chat_template | llm

In [52]:
chain.invoke({"name": "Langchain-Bot", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Langchain-Bot입니다. 어떻게 도와드릴까요?'

### MessagePlaceholder

In [56]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name = "conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)

chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [57]:
# conversation 대화 목록을 나중에 추가하고자 할 떄 MessagesPlaceholder 사용
formatted_chat_prompt = chat_prompt.format(
    word_count = 5,
    conversation = [
        ("human", "안녕하세요! 저는 오늘 새로 입사한 정입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁드립니다.")
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 정입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [58]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [59]:
# chain 실행 및 확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
            "human", "안녕하세요! 저는 오늘 새로 입사한 정입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁드립니다."),
        ],
    }
)

'새로 입사한 정, 만나서 반가워요.'

# 2-2. 퓨샷 프롬프트

### FewShotPromptTemplate

In [61]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature = 0, # 창의성 (0: 보수적, 1: 창의적)
    model_name = "gpt-4-turbo" # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
answer = llm.stream(question)
stream_response(answer)

대한민국의 수도는 서울입니다.

In [62]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# few-shot 예제 데이터 생성

examples = [
    {
        "question": "스티브잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해에 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
"""
    },
    {
        
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

In [63]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
스티브잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [65]:
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Question:\n{question}\nAnswer:",
    input_variables = ["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question = question)
print(final_prompt)

Question:
스티브잡스와 아인슈타인 중 누가 더 오래 살았나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인


Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
율곡 이이의 어머니가 태어난 해에 통치하던 왕은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군


Question:
올드보이와 기생충의 감독이 같은 나라 출신인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [67]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 답변은: 1998년에 Bill Gates의 나이는 43세입니다.

In [69]:
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Question:\n{question}\nAnswer:",
    input_variables = ["question"],
)

#chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google은 언제 창립되었나요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 답변은: 1998년에 Bill Gates의 나이는 43세입니다.

### Example Selector

In [70]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Vector DB 생성
chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 선택 가능한 예시 목록 존재
    examples,
    # 의미적 유사성 측정에 사용되는 임베딩을 생성하는 임베딩 클래스 존재
    OpenAIEmbeddings(),
    # 임베딩 저장, 유사성 검색 수행에 사용되는 VectorStore 클래스 존재
    Chroma,
    # 생성할 예시의 수
    k = 1,
)

# 입력과 가장 유사한 예시 선택
selected_examples = example_selector.select_examples({"question": question})

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
print(f"입력에 가장 유사한 예시: \n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

입력에 가장 유사한 예시: 
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



***FewShotPromptTemplate 생성***
- 기존 FewShotPromptTemplate과 달리 모든 예제를 넣지 않고, Example Selector를 활용
- 질문과 관련 있는 예제만 포함

In [71]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector, # 유사한 예제만 선택
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

# 체인 생성
chain = prompt | llm

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [72]:
# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)

stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google이 창립된 연도는 언제인가요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates는 언제 태어났나요?
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.
최종 계산: 1998년 - 1955년 = 43살
최종 답변은: 43살

### FewShotChatMessagePromptTemplate

In [73]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [74]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings())

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [75]:
# fewshot 예제와 example selector를 사용하여 유사한 예제 1개를 선택
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

example_selector.select_examples(question)

[{'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요'}]

In [77]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a helpful assistant.",
        ),
        few_shot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [78]:
# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 최현수 프로젝트 매니저가 회의를 개회하고, 회의의 목적을 설명함.
   - 목적: 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤에 대한 계획 수립.

2. 프로젝트 진행 상황 보고
   - 황지연 개발자가 개발 진행 상황에 대해 보고.
     - 현재 개발 중인 기능들의 진행 상태와 발생한 문제점에 대해 설명.
     - 해결 방안과 필요한 리소스에 대해 논의.
   - 김태영 디자이너가 UI/UX 디자인 진행 상황에 대해 보고.
     - 최근 완료된 디자인 리뷰와 사용자 피드백에 대한 내용 공유.
     - 다음 단계의 디자인 계획 발표.

3. 다가오는 마일스톤 계획
   - 최현수가 다음 마일스톤의 주요 목표와 일정을 제시.
   - 각 팀원의 역할과 책임에 대해 구체적으로 논의.

4. 목표 설정
   - 팀은 다음 주까지의 구체적인 목표를 설정.
     - 황지연은 주요 기능 개발 완료를 목표로 함.
     - 김태영은 사용자 인터페이스의 최종 검토를 목표로 함.

5. 문제 해결 논의
   - 팀원들이 현재 직면한 문제에 대해 논의하고, 해결책을 모색함.
   - 필요한 추가 리소스나 지원에 대해 최현수와 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 최현수가 담당.

회의는 모든 안건이 충분히 논의되고 목표가 설정된 후 종료됨.

### Example Selector의 유사도 검색 문제 해결 -> 여기부터 한 번 쭉 훑어보기

유사도 계산시 `instruction` 과 `input` 을 사용하고 있습니다. 하지만, `instruction` 만 사용하여 검색시 제대로된 유사도 결과가 나오지 않습니다. 

이를 해결하기 위해 커스텀 유사도 계산을 위한 클래스를 정의합니다.

아래는 잘못 검색된 결과의 예시입니다.

In [79]:
question = {
    "instruction": "회의록을 작성해 주세요",
}

example_selector.select_examples(question)

[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

In [80]:
# 커스텀 하지 않은 기본 예제 선택기를 사용했을 때 결과
example_selector.select_examples({"instruction": "다음 문장을 요약해 주세요"})

[{'answer': '본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.',
  'input': '우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.',
  'instruction': '당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요'}]

In [81]:
from langchain_teddynote.prompts import CustomExampleSelector

# 커스텀 예제 선택기 생성
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

# 커스텀 예제 선택기를 사용했을 때 결과
custom_selector.select_examples({"instruction": "다음 문장을 회의록 작성해 주세요"})

[{'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n'}]

In [82]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  # 예제 프롬프트 사용
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [83]:
# chain 을 생성합니다.
chain = custom_prompt | llm

In [84]:
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

회의록: ABC 기술 회사 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 최현수 프로젝트 매니저가 회의를 개회하고, 회의의 목적을 설명함.
   - 목적: 프로젝트의 현재 진행 상황 검토 및 다가오는 마일스톤에 대한 계획 수립.

2. 프로젝트 진행 상황 보고
   - 황지연 개발자가 개발 진행 상황에 대해 보고.
     - 현재 개발 중인 기능들의 상태와 최근 해결된 문제점들에 대해 설명.
     - 다음 주까지 완료 목표로 설정된 기능들을 공유.
   - 김태영 디자이너가 UI/UX 디자인 진행 상황에 대해 보고.
     - 최근 완성된 디자인 요소들과 사용자 피드백을 바탕으로 한 개선 사항들을 공유.
     - 다음 단계의 디자인 작업 계획을 발표.

3. 마일스톤 계획 논의
   - 최현수가 다가오는 마일스톤에 대한 전체적인 계획을 제시.
   - 팀원들과 함께 다음 주까지의 구체적인 목표와 업무 분담에 대해 논의.

4. 문제점 및 해결 방안 논의
   - 팀원들이 현재 직면한 문제점들을 공유하고, 해결 방안에 대해 토론.
   - 필요한 자원과 지원에 대해 논의.

5. 다음 회의 일정 및 기타 사항
   - 다음 주간 진행 상황 회의 일정을 확정.
   - 기타 통지 사항이나 주의 사항 공유.

6. 마무리
   - 최현수 매니저가 회의를 마무리하며, 팀원들의 노력에 감사를 표함.
   - 회의록은 김태영 디자이너가 정리하여 팀원들에게 배포하기로 함.

In [85]:
question = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

문서 요약: 2023년 글로벌 경제 전망 보고서

- 현재 상태: 세계 경제의 현재 상황에 대한 분석을 제공하며, 경제 회복의 진행 상황과 주요 도전 과제를 설명.
- 경제 성장률: 주요 국가들의 경제 성장률을 검토하고, 각 국가의 경제 상황에 대한 상세한 정보를 제공.
- 글로벌 무역: 현재의 글로벌 무역 동향을 분석하고, 무역 정책 변화와 그 영향을 평가.
- 경제 예측: 2024년에 대한 경제 예측을 제시하며, 성장 전망과 잠재적 위험 요소를 포함.
- 영향 요인: 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 심층 분석.

이 보고서는 다가오는 해의 글로벌 경제 상황에 대한 포괄적인 이해를 제공하며, 다양한 요인들이 경제에 미치는 영향을 평가하여 정책 입안자와 경제 분석가들에게 유용한 정보를 제공합니다.

In [86]:
question = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

회사는 올해 매출이 증가할 것으로 예상하고 있다. 새로운 전략이 효과적으로 작동하고 있기 때문이다.